In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 640 ms (started: 2021-08-26 16:44:35 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #6  Aug 26, 2021'

#git push origin master
git push

In [2]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 5.38 s (started: 2021-08-26 16:46:28 +08:00)


# 设置

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

time: 1 ms (started: 2021-08-26 16:46:45 +08:00)


# 介绍
`masking`是一种告诉序列处理层输入中的某些时间步丢失的方法，因此在处理数据时应该跳过。

`padding`是一种特殊形式的掩码，其中掩码步骤位于序列的开头或结尾。 填充来自需要将序列数据编码成连续的批次：为了使批次中的所有序列适合给定的标准长度，需要填充或截断一些序列。

让我们仔细看看。

# 填充序列数据
在处理序列数据时，单个样本的长度不同是很常见的。 考虑以下示例（文本标记为单词）：

In [5]:
[
  ["Hello", "world", "!"],
  ["How", "are", "you", "doing", "today"],
  ["The", "weather", "will", "be", "nice", "tomorrow"],
]

[['Hello', 'world', '!'],
 ['How', 'are', 'you', 'doing', 'today'],
 ['The', 'weather', 'will', 'be', 'nice', 'tomorrow']]

time: 12.1 ms (started: 2021-08-26 16:49:57 +08:00)


词汇查找后，数据可能被向量化为整数，例如：

In [ ]:
[
  [71, 1331, 4231]
  [73, 8, 3215, 55, 927],
  [83, 91, 1, 645, 1253, 927],
]

数据是一个嵌套列表，其中单个样本的长度分别为 3、5 和 6。 由于深度学习模型的输入数据必须是单个张量（在这种情况下形状为 (batch_size, 6, vocab_size)），比最长项目短的样本需要填充一些占位符值（或者，一个 也可能在填充短样本之前截断长样本）。

Keras 提供了一个实用函数来截断和填充 Python 列表到一个公共长度：`tf.keras.preprocessing.sequence.pad_sequences`。

In [7]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

# 默认情况下，这将使用 0s 填充； 它可以通过“value”参数进行配置。
# 请注意，您可以“预”填充（在开头）或“后”填充（在结尾）。
# 我们建议在使用 RNN 层时使用“post”填充（为了能够使用层的 CuDNN 实现）。
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]
time: 2.78 ms (started: 2021-08-26 16:53:57 +08:00)


# Masking
现在所有样本都有统一的长度，模型必须被告知数据的某些部分实际上是填充并且应该被忽略。 这种机制就是掩蔽。

在 Keras 模型中引入输入掩码有以下三种方式：
* 添加 keras.layers.Masking 层。
* 使用 mask_zero=True 配置 keras.layers.Embedding 层。
* 在调用支持此参数的层（例如 RNN 层）时手动传递掩码参数。

# 掩码生成层：Embedding和Masking
在幕后，这些层将创建一个掩码张量（形状为 (batch, sequence_length) 的二维张量），并将其附加到由掩码或嵌入层返回的张量输出。

In [9]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)
time: 1.84 s (started: 2021-08-26 16:58:25 +08:00)


In [10]:
masking_layer = layers.Masking()
# 通过将 2D 输入扩展为 3D 来模拟嵌入查找，嵌入维度为 10。
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)
time: 21 ms (started: 2021-08-26 17:00:46 +08:00)


从打印结果中可以看出，掩码是一个形状为`(batch_size, sequence_length)` 的 2D 布尔张量，其中每个单独的 False 条目表示在处理过程中应忽略相应的时间步长。

# 功能 API 和顺序 API 中的掩码传播
使用 Functional API 或 Sequential API 时，由 Embedding 或 Masking 层生成的掩码将通过网络传播到能够使用它们的任何层（例如，RNN 层）。 Keras 将自动获取与输入对应的掩码，并将其传递给任何知道如何使用它的层。

例如，在下面的 Sequential 模型中，LSTM 层将自动接收一个掩码，这意味着它将忽略填充值：

In [11]:
model = keras.Sequential(
    [layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True), layers.LSTM(32),]
)

time: 1.18 s (started: 2021-08-26 17:03:02 +08:00)


以下 Functional API 模型也是这种情况：

In [12]:
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

time: 1.3 s (started: 2021-08-26 17:03:22 +08:00)


# 将掩码张量直接传递给图层
可以处理掩码的层（例如 LSTM 层）在其 `__call__` 方法中有一个掩码参数。

同时，生成掩码的层（例如嵌入）公开了一个您可以调用的 `compute_mask(input, previous_mask)` 方法。

因此，您可以将掩码生成层的 `compute_mask()` 方法的输出传递给掩码消耗层的 `__call__` 方法，如下所示：

In [14]:
class MyLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
        self.lstm = layers.LSTM(32)

    def call(self, inputs):
        x = self.embedding(inputs)
        # 请注意，您也可以手动准备一个 `mask` 张量。
        # 它只需要是一个布尔张量
        # 使用正确的形状，即 (batch_size, timesteps)。
        mask = self.embedding.compute_mask(inputs)
        output = self.lstm(x, mask=mask)  # 该层将忽略屏蔽值
        return output


layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype("int32")
layer(x)

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[ 2.10512639e-03,  7.91605655e-03,  2.91242613e-03, ...,
         4.98896092e-03,  1.13983441e-03,  2.61795532e-04],
       [ 2.21813447e-03, -1.06557585e-04,  8.73247348e-03, ...,
        -6.83238730e-03, -3.40499380e-03,  5.75671019e-03],
       [-6.44142088e-03,  1.45612126e-02,  7.02295825e-03, ...,
         3.50034097e-04,  1.32520909e-05, -9.08983347e-04],
       ...,
       [-1.18978610e-02, -6.21891534e-03, -6.85542997e-04, ...,
         3.14077246e-03,  4.66374028e-03,  6.02304656e-03],
       [ 2.04775925e-03, -5.54631464e-03,  7.60610308e-03, ...,
         3.86148854e-03, -1.55524036e-03,  7.93630164e-03],
       [ 5.74535131e-03, -8.79379455e-03,  5.86576853e-03, ...,
        -3.53310985e-04,  6.32018549e-03,  1.16598709e-02]], dtype=float32)>

time: 2.29 s (started: 2021-08-26 17:05:23 +08:00)


# 支持自定义图层中的遮罩
有时，您可能需要编写生成蒙版的图层（如嵌入），或需要修改当前蒙版的图层。

例如，任何生成时间维度与其输入不同的张量的层，例如在时间维度上连接的 `Concatenate` 层，都需要修改当前掩码，以便下游层能够正确地将掩码时间步长转换为 帐户。

为此，您的图层应实现 `layer.compute_mask()` 方法，该方法在给定输入和当前掩码的情况下生成一个新掩码。

这是一个需要修改当前蒙版的 `TemporalSplit` 图层的示例。

In [15]:
class TemporalSplit(keras.layers.Layer):
    """将输入张量沿时间维度拆分为 2 个张量。"""

    def call(self, inputs):
        # 期望输入为 3D，掩码为 2D，沿时间轴（轴 1）将输入张量分成 2 个子张量。
        return tf.split(inputs, 2, axis=1)

    def compute_mask(self, inputs, mask=None):
        # 如果出现，也将mask分成 2 个。
        if mask is None:
            return None
        return tf.split(mask, 2, axis=1)


first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

tf.Tensor(
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]], shape=(3, 3), dtype=bool)
tf.Tensor(
[[False False False]
 [ True  True False]
 [ True  True  True]], shape=(3, 3), dtype=bool)
time: 8.78 ms (started: 2021-08-26 17:07:27 +08:00)


这是一个 CustomEmbedding 层的另一个示例，它能够从输入值生成掩码：

In [16]:
class CustomEmbedding(keras.layers.Layer):
    def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
        super(CustomEmbedding, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.mask_zero = mask_zero

    def build(self, input_shape):
        self.embeddings = self.add_weight(
            shape=(self.input_dim, self.output_dim),
            initializer="random_normal",
            dtype="float32",
        )

    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)

    def compute_mask(self, inputs, mask=None):
        if not self.mask_zero:
            return None
        return tf.not_equal(inputs, 0)


layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype("int32")

y = layer(x)
mask = layer.compute_mask(x)

print(mask)

tf.Tensor(
[[False False  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True False  True  True  True]
 [ True False  True  True  True  True False  True  True False]], shape=(3, 10), dtype=bool)
time: 17.7 ms (started: 2021-08-26 17:09:01 +08:00)


# 选择在兼容层上屏蔽传播
大多数图层不修改时间维度，因此不需要修改当前蒙版。 然而，他们可能仍然希望能够将当前掩码不改变地传播到下一层。 这是一种选择加入的行为。 默认情况下，自定义层将销毁当前掩码（因为框架无法判断传播掩码是否安全）。

如果您有一个不修改时间维度的自定义层，并且您希望它能够传播当前输入掩码，则应在层构造函数中设置 `self.supports_masking = True`。 在这种情况下，`compute_mask()` 的默认行为是只传递当前掩码。

下面是一个被列入蒙版传播白名单的图层示例：

In [17]:
class MyActivation(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyActivation, self).__init__(**kwargs)
        # Signal that the layer is safe for mask propagation
        self.supports_masking = True

    def call(self, inputs):
        return tf.nn.relu(inputs)

time: 514 µs (started: 2021-08-26 17:09:45 +08:00)


您现在可以在蒙版生成层（如嵌入）和蒙版消耗层（如 LSTM）之间使用此自定义图层，它将传递蒙版，使其到达蒙版消耗层。

In [18]:
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
x = MyActivation()(x)  # Will pass the mask along
print("Mask found:", x._keras_mask)
outputs = layers.LSTM(32)(x)  # Will receive the mask

model = keras.Model(inputs, outputs)

Mask found: KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.bool, name=None), name='Placeholder_1:0')
time: 1.04 s (started: 2021-08-26 17:10:23 +08:00)


# 编写需要mask信息的图层
一些层是掩码消费者：他们在调用中接受`mask`参数并使用它来确定是否跳过某些时间步骤。

要编写这样的层，您只需在调用签名中添加一个 `mask=None` 参数即可。 与输入关联的掩码将在可用时传递到您的图层。

下面是一个简单的示例：在输入序列的时间维度（轴 1）上计算 softmax 的层，同时丢弃屏蔽的时间步长。

In [19]:
class TemporalSoftmax(keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_float_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        inputs_exp = tf.exp(inputs) * broadcast_float_mask
        inputs_sum = tf.reduce_sum(inputs * broadcast_float_mask, axis=1, keepdims=True)
        return inputs_exp / inputs_sum


inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
x = layers.Dense(1)(x)
outputs = TemporalSoftmax()(x)

model = keras.Model(inputs, outputs)
y = model(np.random.randint(0, 10, size=(32, 100)), np.random.random((32, 100, 1)))

time: 133 ms (started: 2021-08-26 17:11:43 +08:00)


# 概括
这就是您需要了解的有关 Keras 中的填充和遮罩的全部信息。 回顾一下：
* “masking”是层如何知道何时跳过/忽略序列输入中的某些时间步。
* 有些层是掩码生成器：`Embedding`可以从输入值（如果 mask_zero=True）生成掩码，`masking`层也可以。
* 有些层是掩码消费者：他们在他们的 `__call__ `方法中公开了一个掩码参数。 这就是 RNN 层的情况。
* 在 Functional API 和 Sequential API 中，掩码信息是自动传播的。
* 以独立方式使用图层时，您可以手动将遮罩参数传递给图层。
* 您可以轻松编写修改当前蒙版、生成新蒙版或使用与输入关联的蒙版的图层。